In [11]:
import numpy as np
import pandas as pd
from tqdm import tqdm

train_data=pd.read_csv('Polynomialdata_train.csv')
test_data=pd.read_csv('Polynomialdata_test.csv')
Xtrain=train_data.iloc[0:50000,1:4]
ytrain=train_data.iloc[0:50000,0:1]
Xtest=test_data.iloc[0:10000,1:4]
ytest=test_data.iloc[0:10000,0:1]
X_train=Xtrain.to_numpy()
y_train=ytrain.to_numpy()
X_test=Xtest.to_numpy()
y_test=ytest.to_numpy()

def normalize_data(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    return (X - mean) / std

X_train = normalize_data(X_train)
X_test = normalize_data(X_test)

# Specify the degree of the polynomial
degree = 3  # Set the desired degree to 3

# Initialize X_poly with the original features
X_poly = X_train.copy()

# Loop to generate polynomial features up to the specified degree
for d in range(2, degree + 1):
    # Include terms up to the third degree
    X_poly = np.concatenate((X_poly, X_train**d), axis=1)

# Include specific cross-product terms
num_features = X_train.shape[1]

# terms of the form x1*x2, x1*x3, x2*x3
for i in range(num_features):
    for j in range(i + 1, num_features):
        cross_product = X_train[:, i] * X_train[:, j]
        X_poly = np.concatenate((X_poly, cross_product.reshape(-1, 1)), axis=1)


# terms of the form x1*x2_sq, x1*x3_sq, x2*x3_sq

for i in range(num_features):
    for j in range(num_features):
        if i != j:
            cross_product = X_train[:, i] * X_train[:, j]**2
            X_poly = np.concatenate((X_poly, cross_product.reshape(-1, 1)), axis=1)

x1_x2_x3 = X_train[:, 0] * X_train[:, 1] * X_train[:, 2]
X_poly = np.concatenate((X_poly, x1_x2_x3.reshape(-1, 1)), axis=1)

# Add a constant term for the bias
X_poly_train = np.concatenate((np.ones((X_train.shape[0], 1)), X_poly), axis=1)

In [2]:
# Specify the degree of the polynomial
degree = 3  # Set the desired degree to 3

# X_poly with the original features
X_poly_test = X_test.copy()

#terms of the form x1_sq,x2_sq,x3_sq
for d in range(2, degree + 1):
    # Include terms up to the third degree
    X_poly_test = np.concatenate((X_poly_test, X_test**d), axis=1)

num_features = X_test.shape[1]

# terms of the form x1*x2, x1*x3, x2*x3
for i in range(num_features):
    for j in range(i + 1, num_features):
        cross_product = X_test[:, i] * X_test[:, j]
        X_poly_test = np.concatenate((X_poly_test, cross_product.reshape(-1, 1)), axis=1)


# terms of the form x1*x2_sq, x1*x3_sq, x2*x3_sq,etc....
for i in range(num_features):
    for j in range(num_features):
        if i != j:
            cross_product = X_test[:, i] * X_test[:, j]**2
            X_poly_test = np.concatenate((X_poly_test, cross_product.reshape(-1, 1)), axis=1)
            
# term of the form x1*x2*x3
x1_x2_x3 = X_test[:, 0] * X_test[:, 1] * X_test[:, 2]
X_poly_test = np.concatenate((X_poly_test, x1_x2_x3.reshape(-1, 1)), axis=1)

# Add a constant term for the bias
X_poly_test = np.concatenate((np.ones((X_test.shape[0], 1)), X_poly_test), axis=1)


In [3]:
X_test

array([[ 1.64475599, -0.57890681,  0.77364179],
       [ 0.27017619,  1.3553876 , -1.72519578],
       [-1.68687549,  1.70351081, -0.26027888],
       ...,
       [ 2.13615098, -0.48130225,  0.68811825],
       [-0.07917619, -0.16429977,  1.02502758],
       [ 0.9068352 ,  0.57365547, -0.29594564]])

In [4]:
X_poly_test

array([[ 1.        ,  1.64475599, -0.57890681, ...,  2.09287298,
         0.25927296, -0.7366311 ],
       [ 1.        ,  0.27017619,  1.3553876 , ..., -0.12593096,
        -3.16931497, -0.6317554 ],
       [ 1.        , -1.68687549,  1.70351081, ..., -0.74063628,
        -0.75531605,  0.74794015],
       ...,
       [ 1.        ,  2.13615098, -0.48130225, ...,  3.13998059,
         0.15940387, -0.70747795],
       [ 1.        , -0.07917619, -0.16429977, ...,  0.00642576,
         0.02767002,  0.0133342 ],
       [ 1.        ,  0.9068352 ,  0.57365547, ..., -0.24337092,
        -0.09738997, -0.15395417]])

In [5]:
(X_poly_train)

array([[ 1.        ,  0.23449913,  1.82041534, ...,  0.09411544,
         5.6717802 ,  0.73061762],
       [ 1.        , -0.23836156,  1.05175271, ..., -0.00604934,
        -0.1177776 ,  0.02669226],
       [ 1.        , -0.19769502, -1.89636525, ..., -0.0273981 ,
        -2.52100092, -0.26281293],
       ...,
       [ 1.        ,  1.3624297 ,  0.35255978, ...,  2.51133432,
         0.16816742,  0.64986507],
       [ 1.        ,  0.48957399,  0.30081577, ...,  0.14815314,
         0.05593394,  0.0910318 ],
       [ 1.        ,  0.55268949, -0.2028049 , ...,  0.32947842,
         0.04436306, -0.12089942]])

In [6]:
from tqdm import tqdm

def normalise_data(X, epsilon=1e-8):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    # Add a small constant to avoid division by zero
    std[std < epsilon] = epsilon
    return (X - mean) / std

X_train_poly = normalise_data(X_poly_train)
X_test_poly=normalise_data(X_poly_test)

class SGDRegressor:
    def __init__(self, learning_rate=0.01, epochs=100, epsilon=np.finfo(np.float64).eps, alpha=0.01):
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.epsilon = epsilon
        self.alpha = alpha

    def fit(self, X_train, y_train):
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])

        for epoch in tqdm(range(self.epochs)):
            for i in range(X_train.shape[0]):
                X_i = X_train[i, :].reshape(1, -1)
                y_i = y_train[i]
                
                y_hat = np.dot(X_i, self.coef_.T) + self.intercept_

                intercept_der = -2 * (y_i - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * (y_i - y_hat) * X_i + 2 * self.alpha * self.coef_
                self.coef_ = self.coef_ - (self.lr * coef_der)

                # Clip coefficients to avoid overflow
                self.coef_ = np.clip(self.coef_, -1e6, 1e6)
                self.intercept_ = np.clip(self.intercept_, -1e6, 1e6)

                # Check for overflow during training
                if np.isnan(self.coef_).any() or np.isinf(self.coef_).any() or np.isnan(self.intercept_).any() or np.isinf(self.intercept_).any():
                    print("Numerical instability detected. Reduce learning rate or check data.")
                    return

        print(self.intercept_, self.coef_)

    def predict(self, X_test):
        return np.dot(X_test, self.coef_.T) + self.intercept_

gdr = SGDRegressor(epochs=1000, learning_rate=0.001, alpha=0.01)

gdr.fit(X_train_poly, y_train)


100%|██████████| 1000/1000 [1:57:38<00:00,  7.06s/it] 

[[985213.80215316]] [[ 1.23387954e-319  3.62765714e+005 -2.69040380e+005  4.34647641e+005
   6.20380681e+005  7.10956485e+005  9.99757635e+005 -1.56759310e+005
   5.25347643e+005  6.76677855e+005  1.94043450e+004  9.98535599e+005
   4.15126349e+005 -2.06159932e+005 -6.90357712e+005  1.68967831e+005
   6.36030255e+004 -8.23626203e+005  6.46229574e+005  2.53280268e+005]]


In [7]:
y_pred1=gdr.predict(X_train_poly)
y_pred1

array([[8898517.77400466],
       [-358650.375928  ],
       [ 446452.88476472],
       ...,
       [2966342.39324616],
       [ 447013.59472462],
       [1302189.44914569]])

In [8]:
y_pred2=gdr.predict(X_test_poly)
y_pred2

array([[ 1391779.77763071],
       [-2153441.26956147],
       [ 3413731.83416406],
       ...,
       [ 1602659.220677  ],
       [  616136.8314009 ],
       [ -353865.19223412]])

In [9]:
def SSM(y_train):
    return np.sum((y_train-np.mean(y_train))**2)
def SSR(y_train,y_pred):
    return np.sum((y_train-y_pred)**2)
    
SSr=SSR(y_train,y_pred1)
SSm=SSM(y_train)
r2_score=1-(SSr/SSm)
r2_score


0.11373968691984537

In [10]:
def SSM(y_train):
    return np.sum((y_train-np.mean(y_train))**2)
def SSR(y_train,y_pred):
    return np.sum((y_train-y_pred)**2)
    
SSr=SSR(y_test,y_pred2)
SSm=SSM(y_train)
r2_score=1-(SSr/SSm)
r2_score

0.9904172063767346